<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/5.다층퍼셉트론_XOR_실습.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다층퍼셉트론 XOR 실습

텐서플로우로 직접 다층퍼셉트론을 구현하여 XOR 연산을 다뤄보겠습니다.

# 라이브러리 임포트

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(678)

import numpy as np

In [2]:
print(tf.__version__)

2.0.0


# 텐서플로우 그래프 정의하기
가장 먼저 입력 레이어를 정의하겠습니다.  
XOR 연산 입력 데이터를 X, 결과값을 y라고 정의하겠습니다.  

XOR 연산을 위해 입력값 X는 아래와 같이 [4,2]의 형태를 갖습니다.
  
[0, 0],  
[0, 1],  
[1, 0],  
[1, 1]  

입력값에 따른 출력값 y는 [4,1]의 형태를 갖습니다.    
[0,1,1,0]  

In [3]:
X = np.array([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y = np.array([0.,1.,1.,0.])

# 모델 생성

In [4]:
model = Sequential()
# 첫번째 히든 레이어
model.add(Dense(units=2,activation='sigmoid',input_dim=2))
# 두번째 히든 레이어
model.add(Dense(units=1,activation='sigmoid'))

In [5]:
# 손실 함수 및 최적화 정의
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [6]:
# 모델 요약
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
None


# 모델 학습

In [7]:
# 모델 학습, 10분 정도 걸립니다. 모델 학습 진행 상황을 보시려면 verbose=1로 수정해주세요.
model.fit(X,y,epochs=50000,batch_size=4,verbose=0)

# 테스트
학습 결과를 통해, [0,0], [1,1]은 0에 가까운 값을 출력하고,  
[0,1], [1,0]은 1에 가까운 값을 출력하는 것을 확인할 수 있습니다.

In [17]:
print(model.predict(X,batch_size=4))

[[0.06190902]
 [0.94067526]
 [0.9409089 ]
 [0.07283714]]


# 첫번째 레이어 파라미터 확인
첫째 노드의 가중치는 3.5567362, 3.5493941, 편향값은 -5.40607입니다.  
둘째 노드의 가중치는 5.79164, 5.742752, 편향값은 -2.329901입니다.

In [18]:
print("first layer weights: ",model.layers[0].get_weights()[0])
print("first layer bias: ",model.layers[0].get_weights()[1])

first layer weights:  [[3.5567362 5.79164  ]
 [3.5493941 5.742752 ]]
first layer bias:  [-5.40607  -2.329901]


# 두번째 레이어 파라미터 확인
두번째 레이어에는 노드가 한개만 있고, 그 노드의 가중치는 -7.801387, 편향값은 -3.3388188입니다.

In [19]:
print("second layer weights: ",model.layers[1].get_weights()[0])
print("second layer bias: ",model.layers[1].get_weights()[1])

second layer weights:  [[-7.801387 ]
 [ 7.3920655]]
second layer bias:  [-3.3388188]


# 직접 계산해보기
학습을 통해 구한 가중치와 편향값을 사용하여, 직접 결과를 계산해보는 실습을 해보겠습니다.  
직접 계산한 결과값이 model.predict와 일치하는 것을 확인할 수 있습니다.

In [20]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [21]:
def get_output(x):
    layer0 = model.layers[0]
    # 첫번째 레이어, 첫번째 노드의 결과값
    layer0_weights, layer0_bias = layer0.get_weights()
    layer0_node0_weights = np.transpose(layer0_weights)[0]
    layer0_node0_bias = layer0_bias[0]
    layer0_node0_output = sigmoid( np.dot( x, layer0_node0_weights ) + layer0_node0_bias )
    # 첫번째 레이어, 두번째 노드의 결과값
    layer0_node1_weights = np.transpose(layer0_weights)[1]
    layer0_node1_bias = layer0_bias[1]
    layer0_node1_output = sigmoid( np.dot( x, layer0_node1_weights ) + layer0_node1_bias )
    # 두번째 레이어 결과값
    layer1 = model.layers[1]
    layer1_weights, layer1_bias = layer1.get_weights()
    layer1_output = sigmoid( np.dot( [layer0_node0_output, layer0_node1_output], layer1_weights ) + layer1_bias )

    print(layer1_output)

In [22]:
get_output([0,0])

0.061909028513168735


In [23]:
get_output([0,1])

0.9406752216593391


In [24]:
get_output([1,0])

0.9409089934995193


In [25]:
get_output([1,1])

0.07283725484720817
